In [12]:
# %matplotlib inline
import pandas as pd
import numpy as np
import easygui
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import collections
import bokeh.charts
import bokeh.charts.utils
import bokeh.io
import bokeh.models
import bokeh.palettes
import bokeh.plotting


In [14]:

def osc(files,timeBase=2.5,proc=collections.OrderedDict(),meanLen=300,prestim=200,scale1=1,
        scale2=100,rollWindow=100,threshold=2,xlim=(0,1300),ylim=(-0.01,0.04),
        lims=True,metaLabel=None,exceptions=None,):
    
    Yoffsets=collections.OrderedDict()
    Xoffsets=collections.OrderedDict()
    area=collections.OrderedDict()
    def exceptionHandler(exceptions,df):
        if exceptions:#handle exception if dict present
            for key, value in exceptions.iteritems():
                if key in file:
#                     print file
#                     print len(df)
                    df=df[value:]#delete 1st n items which are junk
#                     print len(df)
        return df

    def readFile(file):
        df=pd.read_csv(file)
        df=df.ix[:,4]
        df=df.dropna()
#         timeBase=timeBase #seconds/div
        
        return df

    for file in files:
        df=readFile(file)
            
        if 'CH2' in file:

            
            df=exceptionHandler(exceptions,df)
            ind=file.index('CH')
            label=file[ind-2:ind]
            
#             Yoffset=3.5
            Yoffset = df.ix[1:meanLen].mean()
            thresh=(df.diff()>threshold)
            fall=thresh[thresh==True]
#             Xoffset=fall.index[0]
            try:
                Xoffset=fall.index[0]
            except IndexError:
                print "No trigger pulse for",label 
                Xoffset=0
                
            Xoffsets[label]=Xoffset

            df=df-Yoffset
            df=df/scale2
            df=abs(df) #flipping
            df=df.ix[Xoffsets[label]-prestim:]
            df.index/=timeBase*10. #seconds/div*noofdiv
    
            proc['trig_fname'+label]=df
            plt.plot(df,label='trig')

#     plt.plot(df,label='trig')

    # plt.show()



    for file in files:
        df=readFile(file)

#         df=pd.read_csv(file)
#         df=df.ix[:,4]
#         df=df.dropna()
        if 'CH1' in file:
            
            df=exceptionHandler(exceptions,df)

            Yoffset =df[1:meanLen].mean() #1st point is off
            ind=file.index('CH')
            
            label=file[ind-2:ind]
            Yoffsets[label]=Yoffset
            df=df-Yoffset
            df=df/scale1
            try:
                df=df.ix[Xoffsets[label]-prestim:]
            except KeyError:
                Xoffsets[label]=0
                df=df.ix[Xoffsets[label]-prestim:]

            df=df.rolling(window=rollWindow).mean()
            if metaLabel:
                try:
                    label=metaLabel[int(label)]+'_fname'+label
                except KeyError:
                    print "%s key missing"%label
            
            df.index/=timeBase*10. #seconds/div*noofdiv

            proc[label]=df
            df.dropna()
            area[label]=np.trapz(df.iloc[200:800])
            plt.plot(df,label=label)
            

    if lims:
        plt.xlim(xlim)
        plt.ylim(ylim)
        
    proc=pd.DataFrame(proc)
#     proc=proc.dropna()
#     print proc
    print area
    return proc,area

#     plt.legend()
#     plt.show()
    


In [4]:
files08=easygui.fileopenbox(filetypes=['*.CSV'],multiple=True,)

In [21]:
files08=easygui.fileopenbox(filetypes=['*.CSV'],multiple=True,)
df,area=osc(files08,scale1=10,metaLabel=metaLabel,meanLen=250,
            exceptions={'50':140,'45':300,'51':200,'55':275,'82':200,'80':400,'81':100},lims=False)
timeBase=2.5 #seconds/div
df.index/=timeBase*10. #seconds/div*noofdiv
plt.legend()
plt.show()


OrderedDict([('ODP small nozzle 0.5/off_fname08', 6.712283935742966), ('0 0 col_fname09', 11.177661542168671), ('0.5 0 col_fname10', -4.3415407068272973), ('0.5 0 nozzle_fname11', -5.6630680481927662), ('0.5, 0.5 nozzle_fname12', -1.9539499919678731)])


In [7]:
df,area=osc(files08,scale1=10,metaLabel=metaLabel,meanLen=250,
            exceptions={'50':140,'45':300,'55':275,'82':200,'80':400,'81':100},lims=False)
timeBase=2.5 #seconds/div
df.index/=timeBase*10. #seconds/div*noofdiv
plt.legend()
plt.show()


OrderedDict([('ODP 0.5/0.5_fname35', 0.37041094779116646)])


In [19]:
metaLabel={
    35:'ODP 0.5/0.5',
    46:'TYGON 0.5/0.5',
    0:'ODP 0.5/0.5',
    2:'ODP 0.5/off',
    6:'ODP small nozzle 0.5/0.5',
    8:'ODP small nozzle 0.5/off',
    9:'0 0 col',
    10:'0.5 0 col',
    11:'0.5 0 nozzle',
    12:'0.5, 0.5 nozzle',
    13:'0.5 0 insect'
}

In [32]:
# df.to_csv('all.csv')
pd.re

In [8]:
df.describe()

/usr/local/lib/python2.7/dist-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,ODP 0.5/0.5_fname35
count,2400.000000
mean,-0.000044
std,0.005495
min,-0.009916
25%,NaN
50%,NaN
75%,NaN
max,0.014342


In [11]:
from pyqtgraph import examples
examples.run()

In [37]:
import pandas as pd
import numpy as np
from bokeh.palettes import Spectral11
from bokeh.plotting import figure, show, output_file
output_file('temp.html')

# toy_df = pd.DataFrame(data=np.random.rand(5,3), columns = ('a', 'b' ,'c'), index = pd.DatetimeIndex(start='01-01-2015',periods=5, freq='d'))   
toy_df=df
numlines=len(toy_df.columns)
mypalette=Spectral11[0:numlines]

p = figure(width=1000, height=500, x_axis_type="datetime",) 
p.multi_line(xs=[toy_df.index.values]*numlines,
                ys=[toy_df[name].values for name in toy_df],
                line_color=mypalette,
                line_width=5)
show(p)

TypeError: '_list_attr_splat' object is not callable

In [40]:
import pandas as pd
import numpy as np
from bokeh.charts import TimeSeries
from bokeh.models import HoverTool
from bokeh.plotting import show

toy_df = pd.DataFrame(data=np.random.rand(5,3), columns = ('a', 'b' ,'c'), index = pd.DatetimeIndex(start='01-01-2015',periods=5, freq='d'))       
_tools_to_show = 'box_zoom,pan,save,hover,resize,reset,tap,wheel_zoom'        
# toy_df = df

p = figure(width=1200, height=900, x_axis_type="datetime", tools=_tools_to_show)


# FIRST plot ALL lines (This is a hack to get it working, why can't i pass in a dataframe to multi_line?)   
# It's not pretty but it works. 
# what I want to do!: p.multi_line(df)
ts_list_of_list = []
for i in range(0,len(toy_df.columns)):
    ts_list_of_list.append(toy_df.index.T)

vals_list_of_list = toy_df.values.T.tolist()

# Define colors because otherwise multi_line will use blue for all lines...
cols_to_use =  ['Black', 'Red', 'Lime']
p.multi_line(ts_list_of_list, vals_list_of_list, line_color=cols_to_use)


# THEN put  scatter one at a time on top of each one to get tool tips (HACK! lines with tooltips not yet supported by Bokeh?) 
for (name, series) in toy_df.iteritems():
    # need to repmat the name to be same dimension as index
    name_for_display = np.tile(name, [len(toy_df.index),1])

    source = ColumnDataSource({'x': toy_df.index, 'y': series.values, 'series_name': name_for_display, 'Date': toy_df.index.format()})
    # trouble formating x as datestring, so pre-formating and using an extra column. It's not pretty but it works.

    p.scatter('x', 'y', source = source, fill_alpha=0, line_alpha=0.3, line_color="grey")     

    hover = p.select(dict(type=HoverTool))
    hover.tooltips = [("Series", "@series_name"), ("Date", "@Date"),  ("Value", "@y{0.00%}"),]
    hover.mode = 'mouse'

show(p)

NameError: name 'ColumnDataSource' is not defined

In [5]:
metaLabel={20:'0.5 5.8', 21:'1.5 5.8',22:'off off',23:'0.5 5.8',25:'0.5 30',27:'0.5 55',28:'0.5 15', 29:'0.5 off',
          30:'1_lpm',31:'1.5_lpm',32:'0.5_lpm',33:'2_lpm',
           34:'0lpm',45:'splitless',
           46:'split',50:'white obj',52:'cone adjust',57:'split loose',62:'direct',63:'direct gap' ,
           65:'split',
           66:'split',
           70:'flush',
           71:'not flush',
           72:'flush',
           73:'new',
           75:'pipette',
           76:'0.1ul pure',
           77:'0.5s pipette',
           78:'3s pipette',
           80:'pure 0.1ul splitless',
           81:'0.1ul 10E-2 splitless',
           82:'pure 0.1ul split 1:1',
           84:'0.1ul 10E-2  split 1:1'
          }

In [63]:
df.plot()
plt.show()

In [ ]:
df=None

In [ ]:
area

In [ ]:
# df['48'].plot()
# plt.show()
# np.trapz(df['48']
np.trapz(df['48'].iloc[260:1000])-df['48'].iloc[250:1000].sum()

In [ ]:
df['48'].iloc[250:1000].sum()

In [ ]:
p = bokeh.charts.Line(df)

# Display it
bokeh.io.show(p)

In [ ]:
from bokeh.charts import Histogram, output_file, show
from bokeh.sampledata.autompg import autompg as df

p = Histogram(df, values='hp', color='cyl',
              title="HP Distribution (color grouped by CYL)",
              legend='top_right')

output_file("histogram_color.html")

show(p)

In [ ]:
files31 = easygui.fileopenbox(filetypes=['*.CSV'],multiple=True)
files01 = easygui.fileopenbox(filetypes=['*.CSV'],multiple=True)
files01h = easygui.fileopenbox(filetypes=['*.CSV'],multiple=True)


In [ ]:
#13 15
df=osc(files31)

#20 27
df=osc(files01,scale1=10,metaLabel=metaLabel,proc=df)

df=osc(files01h,scale1=10,metaLabel=metaLabel,proc=df)

timeBase=5 #seconds/div
df.index/=timeBase*10. #seconds/div*noofdiv
plt.legend()
plt.show()
df.index

In [ ]:
df.to_csv('test.csv')

In [ ]:
158+180+174+232+370+366+380+171+469+269+133+468+206+190+400+275+158+200+190+100+213+363+420+221+290+127+249+115+225+200+284


In [ ]:
554+633+570+776+728+791+765+733+716+949+621

In [ ]:
3893+744+131+8825+2585+73+486

In [ ]:
4930+4424+4039+4897+3428+3661

In [ ]:
2075+1042+1076+1259+2016

In [ ]:
422+4917+2459+1934+273+390+569+2595+1484+569+991+829+219+917+1424+1730

In [ ]:
7796+7836+16737+25379+7468+21722

In [ ]:
250+300+500+

In [23]:
import numpy as np

In [27]:
b=1
r=range(100)


In [32]:
t=2*np.arctan(b/(2.0*np.array(r)))

In [33]:
plt.plot(r,t)
plt.show()

In [8]:
'amazon'
# 12762+4058+1364+1465
640+5049+6433-1080+12762+4058+1364+1465
12762+4058+1364+6433-1080+1465+5049+640

30691

In [3]:
'om'
422+4917+2459+1934+273+390+569+2595+1484+569+991+829+219+917+1424+1730+733+158

22613

In [6]:
'misc big'
4897+5546+6654+3499

20596

In [4]:
'misc 500+'
400+468+728+949+469+621+776+716+765+791+633

7316

In [5]:
'misc --500-'
174+180+127+140+171+380+370+366+232+115+225+200+363+213+100+190+200+158+275+190+206

4575

In [7]:
290+221+420+284

1215